# Filter barcodes to remove ones aligning to multiple inserts or second barcode

Arvind Rasi Subramaniam

27 Oct 2021

**Edit this Rscript only in the accompanying .ipynb file. The `snakemake` workflow will automatically export it as a .R script.**

## Load libraries

In [66]:
library(Biostrings)
library(GenomicAlignments)
library(plyranges)
library(tidyverse)

## Define analysis-specific variables

In [67]:
args <- commandArgs(trailingOnly = T)
barcode1_alignment_file <- args[1]
barcode_insert_file <- args[2]
read_count_cutoff <- args[3]
output_file <- args[4]

## Read insert-barcode pair counts 

In [68]:
insert_barcodes <- read_csv(barcode_insert_file) %>% 
  rename(read_count = count) %>%
  filter(read_count >= as.integer(read_count_cutoff)) %>%
  mutate(barcode_num = str_c(barcode_num, '_', sample)) %>%
  arrange(barcode_num) %>%
  print()

Rows: 75741 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): barcode, sample
dbl (3): insert_num, barcode_num, count

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 75,741 × 5
   insert_num barcode_num barcode                  sample read_count
        <dbl> <chr>       <chr>                    <chr>       <dbl>
 1         77 10_89lib1   TGGACTGCCCACCCCGGTGGCTCT 89lib1        741
 2        174 10_89lib2   AGTTATGATATGCGCTGGTATTGT 89lib2        757
 3         58 10_89lib4   GCCGGTCCGTCTAGTCATGTGTGG 89lib4        545
 4        222 10_89lib5   GTGGAGTCTAGCTTTGAGTGTAGG 89lib5        257
 5        109 10_89lib6   AGTTTTACTTAGGCCGCGACGGTC 89lib6        587
 6         99 100_89lib1  GATGGTTCGTGTCAGGCCTTCATC 89lib1        546
 7        180 100_89lib2  TGGGTGATTAGGCAGGGTGGTATC 89lib2        552
 8        234 100_89lib3  TATTGGTTGCACTGGCTGGGGATT 89lib3        440
 9        195 100_89lib4  CTTACGTGTTGCAATTTGGAGGGC 89lib4        397
10        163 100_89lib5  GTGATCAGGCATCGCAATGGGGGG 89lib5        193
# … with 75,731 more rows


## How many barcode_1 have multiple inserts?

In [69]:
many_to_one_barcode_combinations <- insert_barcodes %>% 
  group_by(barcode) %>% 
  mutate(n1 = dplyr::n()) %>% 
  ungroup() %>% 
  filter(n1 > 1) %>% 
  print()

# A tibble: 0 × 6
# … with 6 variables: insert_num <dbl>, barcode_num <chr>, barcode <chr>,
#   sample <chr>, read_count <dbl>, n1 <int>


## Fields to read from BAM file

In [70]:
# extract the number of mismatches and total edits
param <- ScanBamParam(
  # what = scanBamWhat(),
  what = c("qname", "flag"),
  # extract number of mismatches
  tag = c("XM"), 
  # include only snps; exclude indels
  simpleCigar = T
)

## Read barcode vs barcode alignments for barcodes 1

In [71]:
bamfile1 <- BamFile(barcode1_alignment_file)
alns1 <- readGAlignments(bamfile1, param = param) %>% 
  as_tibble() %>% 
  mutate(rname = as.character(seqnames)) %>% 
  select(rname, qname, flag, XM) %>% 
  type_convert() %>% 
  print()


── Column specification ────────────────────────────────────────────────────────
cols(
  rname = col_character(),
  qname = col_character()
)



# A tibble: 154,225 × 4
   rname    qname         flag    XM
   <chr>    <chr>        <int> <int>
 1 2_89lib1 2_89lib1         0     0
 2 2_89lib1 6691_89lib1    256     1
 3 2_89lib1 12020_89lib1   256     1
 4 2_89lib1 12022_89lib1   256     1
 5 2_89lib1 12023_89lib1   256     1
 6 3_89lib1 3_89lib1         0     0
 7 3_89lib1 5921_89lib1    256     1
 8 3_89lib1 5923_89lib1    256     1
 9 3_89lib1 7594_89lib1    256     1
10 3_89lib1 7596_89lib1    256     1
# … with 154,215 more rows


## Find barcode_1 that are linked to distinct insert or might be sequencing errors

In [72]:
exclude1 <- alns1 %>% 
  filter(rname != qname) %>%
  left_join(select(insert_barcodes, insert_num, barcode_num, read_count), by = c("rname" = "barcode_num")) %>%
  rename(rinsert = insert_num, rcount = read_count) %>%
  right_join(select(insert_barcodes, insert_num, barcode_num, read_count), by = c("qname" = "barcode_num")) %>%
  rename(qinsert = insert_num, qcount = read_count) %>%
  # this exludes:
  # 1. barcodes that map to two distinct inserts
  # 2. barcodes that got lower count than another homologous barcode with same insert
  filter(!(qinsert == rinsert & qcount > rcount)) %>%
  arrange(qname) %>% 
  distinct(qname) %>%
  print()

# A tibble: 32,563 × 1
   qname       
   <chr>       
 1 10000_89lib1
 2 10000_89lib2
 3 10000_89lib3
 4 10000_89lib4
 5 10000_89lib5
 6 10000_89lib6
 7 10000_89lib7
 8 10001_89lib1
 9 10001_89lib2
10 10001_89lib3
# … with 32,553 more rows


## Write barcodes that do not clash to output

In [76]:
filtered_barcodes <- insert_barcodes %>% 
  anti_join(select(exclude1, qname), by = c("barcode_num" = "qname")) %>%
  anti_join(select(many_to_one_barcode_combinations, barcode_num), by = "barcode_num") %>%
  select(insert_num, barcode_num, barcode, read_count, sample) %>%
  arrange(desc(read_count)) %>%
  rename(linkage_count = read_count) %>%
  mutate(barcode_num = 1:dplyr::n()) %>%
  write_csv(output_file) %>%
  print()

# A tibble: 43,178 × 5
   sample insert_num barcode_num barcode                  linkage_count
   <chr>       <dbl>       <int> <chr>                            <dbl>
 1 89lib2         99           1 GAGGGCTTTGAGGCTGACTGTGGG           843
 2 89lib2        200           2 GGCGAGTACGGGGCCAGTGTTCTC           830
 3 89lib1        188           3 GGTTAGGACCACGGCATGTGCAGT           821
 4 89lib2        238           4 GTGGCCTGTTCCTAGCGCGCCGCT           815
 5 89lib2         92           5 GACAATGAGCGGGACGGGTTTAAT           814
 6 89lib2        201           6 GATGACCTGGTTGCTATGTGCGGT           792
 7 89lib2        222           7 GGGCATTGTGCTAAGTATGGGATG           784
 8 89lib1         43           8 TTGACGCCCTGGTGTTGTCGTGGG           778
 9 89lib1         21           9 GAGTGCGTGATGAGGTGGGGGGGT           777
10 89lib2         34          10 GTTTAGAAGCTGTCCTATACGGTT           762
# … with 43,168 more rows
